In [2]:
import sys
import os
import yaml
from datetime import datetime
import time
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient, Input
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component
from azure.ai.ml import command
from azure.ai.ml.entities import Data, Environment, BuildContext
from azure.ai.ml.entities import Model
from azure.ai.ml import Input
from azure.ai.ml import Output
from azure.ai.ml.constants import AssetTypes
from azure.core.exceptions import ResourceNotFoundError, ResourceExistsError
from azureml.core import Workspace, Run


In [3]:
with open('./llama-fc_config.yaml') as f:
    d = yaml.load(f, Loader=yaml.FullLoader)
    
AZURE_SUBSCRIPTION_ID = d['config']['AZURE_SUBSCRIPTION_ID']
AZURE_RESOURCE_GROUP = d['config']['AZURE_RESOURCE_GROUP']
AZURE_WORKSPACE = d['config']['AZURE_WORKSPACE']
AZURE_DATA_NAME = d['config']['AZURE_SFT_DATA_NAME']    
DATA_DIR = d['config']['SFT_DATA_DIR']
CLOUD_DIR = d['config']['CLOUD_DIR']
HF_MODEL_NAME_OR_PATH = d['config']['HF_MODEL_NAME_OR_PATH']


In [4]:
credential = DefaultAzureCredential()
ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    ml_client = MLClient(credential, AZURE_SUBSCRIPTION_ID, AZURE_RESOURCE_GROUP, AZURE_WORKSPACE)

def get_or_create_model_asset(ml_client, model_name, experiment_name = None, job_name = None, model_dir="outputs", model_type="custom_model", update=False):
    
    try:
        latest_model_version = max([int(m.version) for m in ml_client.models.list(name=model_name)])
        if update:
            raise ResourceExistsError('Found Model asset, but will update the Model.')
        else:
            model_asset = ml_client.models.get(name=model_name, version=latest_model_version)
            print(f"Found Model asset: {model_name}. Will not create again")
    except (ResourceNotFoundError, ResourceExistsError) as e:
        print(f"Exception: {e}") 
        ws = Workspace.from_config()  
  
        # Get the run by its ID   
        run = Run(ws.experiments[experiment_name], job_name)  
        # Register the model  
        model_asset = run.register_model(  
            model_name=model_name,  # this is the name the model will be registered under  
            model_path=model_dir  # this is the path to the model file in the run's outputs  
        )         
        print(f"Created Model asset: {model_name}")

    return model_asset


Found the config file in: /config.json


In [5]:
model = get_or_create_model_asset(ml_client, d['serve']['azure_model_name'], update = False)

Found Model asset: llama-fc-ft. Will not create again


In [31]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import pandas as pd

model_kwargs = dict(
        trust_remote_code=True,    
        device_map={"":0},
        torch_dtype="auto" 
    )
    
model = AutoModelForCausalLM.from_pretrained("./model/outputs", **model_kwargs)

In [7]:
tokenizer = AutoTokenizer.from_pretrained("./model/outputs")

In [8]:
dataset = load_dataset("glaiveai/glaive-function-calling-v2", split="train")
val_dataset = dataset.select(range(2000, 2200))

print(val_dataset[-1])

{'system': 'SYSTEM: You are a helpful assistant with access to the following functions. Use them if required -\n{\n    "name": "create_event",\n    "description": "Create a new event in the calendar",\n    "parameters": {\n        "type": "object",\n        "properties": {\n            "title": {\n                "type": "string",\n                "description": "The title of the event"\n            },\n            "start_time": {\n                "type": "string",\n                "description": "The start time of the event"\n            },\n            "end_time": {\n                "type": "string",\n                "description": "The end time of the event"\n            }\n        },\n        "required": [\n            "title",\n            "start_time",\n            "end_time"\n        ]\n    }\n}\n\n{\n    "name": "generate_random_number",\n    "description": "Generate a random number within a given range",\n    "parameters": {\n        "type": "object",\n        "properties": {\

In [9]:
import json

def parse_conversation(input_string):  
    
    ROLE_MAPPING = {"USER" : "user", "ASSISTANT" : "assistant", "SYSTEM" : "system", "FUNCTION RESPONSE" : "tool"}

    # Regular expression to split the conversation based on SYSTEM, USER, and ASSISTANT  
    pattern = r"(SYSTEM|USER|ASSISTANT|FUNCTION RESPONSE):"  
      
    # Split the input string and keep the delimiters  
    parts = re.split(pattern, input_string)  
      
    # Initialize the list to store conversation entries  
    conversation = []  
      
    # Iterate over the parts, skipping the first empty string  
    for i in range(1, len(parts), 2):  
        role = parts[i].strip()  
        content = parts[i + 1].strip()  
        content = content.replace("<|endoftext|>", "").strip()

        if content.startswith('<functioncall>'):  # build structured data for function call
                # try to turn function call from raw text to structured data
                content = content.replace('<functioncall>', '').strip()
                # replace single quotes with double quotes for valid JSON
                clean_content = content.replace("'{", '{').replace("'}", '}')
                data_json = json.loads(clean_content)
                # Make it compatible with openAI prompt format
                func_call = {'recipient_name': f"functions.{data_json['name']}", 'parameters': data_json['arguments']}
                content = {'tool_uses': [func_call]}
          
        # Append a dictionary with the role and content to the conversation list  
        conversation.append({"role": ROLE_MAPPING[role], "content": content})  
      
    return conversation  

def apply_chat_template(examples):
        conversations = []
        for system, chat in zip(examples["system"], examples["chat"]):
            try:
                system_message = parse_conversation(system)
                chat_message = parse_conversation(chat)
                message = system_message + chat_message
                conversations.append(message)
            except Exception as e:
                print(e) 

        text = [tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=False) for message in conversations]
        return {"text": text}


In [10]:
processed_val_dataset = []
for i in range(len(val_dataset)):
    system_message = parse_conversation(val_dataset[i]["system"])
    chat_message = parse_conversation(val_dataset[i]["chat"])

    message = system_message + chat_message
    processed_val_dataset.append(message)  

In [11]:
## Second-level response
def get_multilevel_qna_pairs(message):
    prompts = []
    answers = []
    is_first_response = False
    no_of_turns = sum([item["role"] == "assistant" for item in message])
    for i, item in enumerate(message):
        if item['role'] == 'assistant':
            prompts.append(message[:i])
            answers.append(item["content"])

    return prompts, answers            

In [14]:
input_data = []
answers = []
for example in processed_val_dataset:
    prompts, responses = get_multilevel_qna_pairs(example)
    input_data.extend(prompts)
    answers.extend(responses)

In [15]:
def get_chat_completion(input_data):
    inputs = tokenizer.apply_chat_template(input_data, tokenize = True, add_generation_prompt = True, return_tensors = "pt").to("cuda")
    outputs = model.generate(input_ids = inputs, max_new_tokens = 1024, do_sample = True, temperature = 0.1)
    response = tokenizer.decode(outputs[0][inputs.shape[-1]:], skip_special_tokens = True)

    return response

In [94]:
# if response[1:12] == "'tool_uses'":
#     predicted_response = ast.literal_eval(response)
#     if isinstance(predicted_response, dict):
#         predicted_functions = [func["recipient_name"] for func in predicted_response["tool_uses"]]
#         predicted_function_args = [func["parameters"] for func in predicted_response["tool_uses"]]

#         actual_response = second_level_responses[-1][0]
#         actual_functions = [func["recipient_name"] for func in actual_response["tool_uses"]]
#         actual_function_args = [func["parameters"] for func in actual_response["tool_uses"]]

#         print(predicted_functions == actual_functions)
#         print(predicted_function_args == actual_function_args)


['functions.create_event']
True
True


In [16]:
import numpy as np
import json
import os
from IPython.display import display
import pandas as pd
import itertools
import time
import base64
from typing import Any, Dict, List, Generator
import ast

In [43]:
!{sys.executable} -m pip install rouge-score

  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 35.3 MB/s eta 0:00:0000:01
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=2d06968994b8441f006041aea19433a8e8567f2102330b09431dc8c927a4baaa
  Stored in directory: /home/azureuser/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [98]:
# from rouge_score import rouge_scorer  

# # Reference and candidate summaries  
# reference = "The cat sat on the mat."  
# candidate = "The cat is sitting on the mat."  

# # Initialize the ROUGE scorer  
# scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)  

# # Calculate ROUGE scores  
# scores = scorer.score(reference, candidate)  

# # Print the scores  
# print("ROUGE-1:", scores['rouge1'].fmesaure)  
# print("ROUGE-2:", scores['rouge2'].fmeasure)  
# print("ROUGE-L:", scores['rougeL'].fmeasure)  

AttributeError: 'Score' object has no attribute 'fmesaure'

In [27]:
from rouge_score import rouge_scorer

def eval(input_data : List, expected_output : List):
    """
    Evaluate the performance of a model in selecting the correct function based on given prompts.

    Args:
        input_data (List) : List of input prompts for evaluation and benchmarking
        expected_output (List) : List of expected response

    Returns:
        df : Pandas Dataframe with input prompts, actual response, expected response, Match/No Match and ROUGE Score
    """
    # Initialize the ROUGE Scorer where llm response is not function-call
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True) 

    # For generic model response without function-call, set a threshold to classify it as a match
    match_threshold_g = 0.75

    function_call_match = []
    function_call_args_match = []
    rouge_fmeasure_score = []
    result_list = []
    latencies = []

    for prompt, answer in zip(input_data, expected_output):

        start_time = time.time()
        predicted_response = get_chat_completion(prompt)
        end_time = time.time()

        actual_response = answer
        is_func_call = False

        latency = (end_time - start_time) * 1000  # convert to milliseconds
        latencies.append(latency)

        if predicted_response[1:12] == "'tool_uses'":
            is_func_call = True
            try:
                predicted_response = ast.literal_eval(predicted_response)
            except:
                continue
            if isinstance(predicted_response, dict):
                predicted_functions = [func["recipient_name"] for func in predicted_response["tool_uses"]]
                predicted_function_args = [func["parameters"] for func in predicted_response["tool_uses"]]

                actual_functions = [func["recipient_name"] for func in actual_response["tool_uses"]]
                actual_function_args = [func["parameters"] for func in actual_response["tool_uses"]]

                fcall_match = predicted_functions == actual_functions
                fcall_args_match = predicted_function_args == actual_function_args
                function_call_match.append(fcall_match)
                function_call_args_match.append(fcall_args_match)
                match = "Yes" if fcall_match and fcall_args_match else "No"
        else:
            fmeasure_score = scorer.score(actual_response, predicted_response)['rougeL'].fmeasure 
            rouge_fmeasure_score.append(fmeasure_score)
            match = "Yes" if fmeasure_score >= match_threshold_g else "No"
        
        result_list.append(
            {
                "Prompt": prompt,
                "Actual": actual_response,
                "Expected": predicted_response,
                "FCall Match": fcall_match if is_func_call else "NA",
                "Fcall_Args_Match": fcall_args_match if is_func_call else "NA",
                "Match": match,
            })


    # Calculate the number of matches
    fcall_matches = sum(function_call_match)
    fcall_match_percentage = (fcall_matches / len(function_call_match)) * 100

    # Calculate the number of correct arguments
    fcall_args_matches = sum(function_call_args_match)
    fcall_args_match_percentage = (fcall_args_matches/ len(function_call_args_match)) * 100

    # Calculate the rouge-f1 score for non-function-call response
    rouge_score_accuracy = sum(rouge_fmeasure_score)/len(rouge_fmeasure_score) * 100

    # Calculate average latency
    avg_latency = sum(latencies) / len(input_data)

    # Create a DataFrame to store the results
    results_df = pd.DataFrame(columns=["Prompt", "Actual", "Expected", "FCall_Match", "FCall_Args_Match", "Match"])
    results_df = pd.DataFrame(result_list)

    def style_rows(row):
        match = row["Match"]
        background_color = "red" if match == "No" else "white"
        return ["background-color: {}; color: black".format(background_color)] * len(
            row
        )

    styled_results_df = results_df.style.apply(style_rows, axis=1)

    # Display the DataFrame as a table
    display(styled_results_df)

    print(
        f"Number of fcall matches: {fcall_matches} out of {len(function_call_match)} ({fcall_match_percentage:.2f}%)"
    )
    print(
        f"Number of fcall args matches: {fcall_args_matches} out of {len(function_call_args_match)} ({fcall_args_match_percentage:.2f}%)"
    )
    print(f"Generic LLM accuracy : {rouge_score_accuracy:.2f}%")
    print(f"Average latency per request: {avg_latency:.2f} ms")

    return results_df

In [30]:
# Evaluate the fine-tuned model accuracy
eval_baseline_df = []
batch_size = 5
for i in range(0, len(input_data), batch_size):
    try:
        output_df = eval(input_data[i : i+batch_size], answers[i: i+batch_size])
        eval_baseline_df.append(output_df)
    except Exception as e:
        print(i)
        continue

,Prompt,Actual,Expected,Match
0,"[{'role': 'system', 'content': 'You are a helpful assistant with access to the following functions. Use them if required -\n{\n ""name"": ""search_books"",\n ""description"": ""Search for books based on title or author"",\n ""parameters"": {\n ""type"": ""object"",\n ""properties"": {\n ""query"": {\n ""type"": ""string"",\n ""description"": ""The search query""\n },\n ""author"": {\n ""type"": ""string"",\n ""description"": ""The author of the book""\n }\n }\n }\n}\n\n{\n ""name"": ""calculate_tip"",\n ""description"": ""Calculate the tip amount for a bill"",\n ""parameters"": {\n ""type"": ""object"",\n ""properties"": {\n ""bill_amount"": {\n ""type"": ""number"",\n ""description"": ""The total bill amount""\n },\n ""tip_percentage"": {\n ""type"": ""number"",\n ""description"": ""The percentage of tip to be added""\n }\n },\n ""required"": [\n ""bill_amount"",\n ""tip_percentage""\n ]\n }\n}'}, {'role': 'user', 'content': ""I am looking for a book but I can't remember the title. The author's name is George Orwell.""}]","{'tool_uses': [{'recipient_name': 'functions.search_books', 'parameters': {'query': '', 'author': 'George Orwell'}}]}","{'tool_uses': [{'recipient_name': 'functions.search_books', 'parameters': {'query': 'George Orwell', 'author': 'George Orwell'}}]}",No
1,"[{'role': 'system', 'content': 'You are a helpful assistant with access to the following functions. Use them if required -\n{\n ""name"": ""search_books"",\n ""description"": ""Search for books based on title or author"",\n ""parameters"": {\n ""type"": ""object"",\n ""properties"": {\n ""query"": {\n ""type"": ""string"",\n ""description"": ""The search query""\n },\n ""author"": {\n ""type"": ""string"",\n ""description"": ""The author of the book""\n }\n }\n }\n}\n\n{\n ""name"": ""calculate_tip"",\n ""description"": ""Calculate the tip amount for a bill"",\n ""parameters"": {\n ""type"": ""object"",\n ""properties"": {\n ""bill_amount"": {\n ""type"": ""number"",\n ""description"": ""The total bill amount""\n },\n ""tip_percentage"": {\n ""type"": ""number"",\n ""description"": ""The percentage of tip to be added""\n }\n },\n ""required"": [\n ""bill_amount"",\n ""tip_percentage""\n ]\n }\n}'}, {'role': 'user', 'content': ""I am looking for a book but I can't remember the title. The author's name is George Orwell.""}, {'role': 'assistant', 'content': {'tool_uses': [{'recipient_name': 'functions.search_books', 'parameters': {'query': '', 'author': 'George Orwell'}}]}}, {'role': 'tool', 'content': '{""results"": [{""title"": ""1984"", ""author"": ""George Orwell""}, {""title"": ""Animal Farm"", ""author"": ""George Orwell""}, {""title"": ""Down and Out in Paris and London"", ""author"": ""George Orwell""}]}'}]","I found several books by George Orwell. Here are a few: ""1984"", ""Animal Farm"", and ""Down and Out in Paris and London"".","I found several books by George Orwell. The titles are ""1984"", ""Animal Farm"", and ""Down and Out in Paris and London"".",Yes
2,"[{'role': 'system', 'content': 'You are a helpful assistant with access to the following functions. Use them if required -\n{\n ""name"": ""search_books"",\n ""description"": ""Search for books based on title or author"",\n ""parameters"": {\n ""type"": ""object"",\n ""properties"": {\n ""query"": {\n ""type"": ""string"",\n ""description"": ""The search query""\n },\n ""author"": {\n ""type"": ""string"",\n ""description"": ""The author of the book""\n }\n }\n }\n}\n\n{\n ""name"": ""calculate_tip"",\n ""description"": ""Calculate the tip amount for a bill"",\n ""parameters"": {\n ""type"": ""object"",\n ""properties"": {\n ""bill_amount"": {\n ""type"": ""number"",\n ""description"": ""The total bill amount""\n },\n ""tip_percentage"": {\n ""type"": ""number"",\n ""description"": ""The percentage of tip to be added""\n }\n },\n ""required"": [\n ""bill_amount"",\n ""tip_percentage""\n ]\n }\n}'}, {'role': 'user', 'content': ""I am looking for a book but I 

Number of fcall matches: 1 out of 1 (100.00%)
Number of fcall args matches: 0 out of 1 (0.00%)
Generic LLM accuracy : 62.25%
Average latency per request: 3302.40 ms
5


,Prompt,Actual,Expected,Match
0,"[{'role': 'system', 'content': 'You are a helpful assistant, with no access to external functions.'}, {'role': 'user', 'content': 'Identify what color a banana would most likely be associated with.'}, {'role': 'assistant', 'content': 'Yellow\n .'}, {'role': 'user', 'content': 'Can you explain why bananas turn brown?'}, {'role': 'assistant', 'content': 'Bananas turn brown due to a process called enzymatic browning, which occurs when enzymes in the fruit react with oxygen from the air. This causes the fruit to darken in color and eventually become soft and mushy.'}, {'role': 'user', 'content': 'How can we prevent bananas from turning brown too quickly?'}]","One way to prevent bananas from turning brown too quickly is to store them in a cool, dry place away from direct sunlight. You can also wrap the stem of the banana bunch in plastic wrap, as this will slow down the ripening process.","To prevent bananas from turning brown too quickly, you can store them in a cool, dry place away from direct sunlight. You can also wrap them in plastic wrap or aluminum foil to prevent oxygen from reaching the fruit. Additionally, you can store them in the refrigerator to slow down the browning process.",No
1,"[{'role': 'system', 'content': 'You are a helpful assistant, with no access to external functions.'}, {'role': 'user', 'content': 'Identify what color a banana would most likely be associated with.'}, {'role': 'assistant', 'content': 'Yellow\n .'}, {'role': 'user', 'content': 'Can you explain why bananas turn brown?'}, {'role': 'assistant', 'content': 'Bananas turn brown due to a process called enzymatic browning, which occurs when enzymes in the fruit react with oxygen from the air. This causes the fruit to darken in color and eventually become soft and mushy.'}, {'role': 'user', 'content': 'How can we prevent bananas from turning brown too quickly?'}, {'role': 'assistant', 'content': 'One way to prevent bananas from turning brown too quickly is to store them in a cool, dry place away from direct sunlight. You can also wrap the stem of the banana bunch in plastic wrap, as this will slow down the ripening process.'}, {'role': 'user', 'content': 'Are there any health benefits associated with eating bananas?'}]","Yes, bananas are considered a healthy food choice as they are a good source of potassium, fiber, vitamin C, and vitamin B6. They may also have other health benefits such as promoting digestion and reducing the risk of heart disease.","Yes, bananas are a good source of several essential nutrients, including potassium, vitamin C, and fiber. They are also relatively low in calories and high in antioxidants, which can help protect against chronic diseases such as heart disease and cancer.",No
2,"[{'role': 'system', 'content': 'You are a helpful assistant, with no access to external functions.'}, {'role': 'user', 'content': 'Identify what color a banana would most likely be associated with.'}, {'role': 'assistant', 'content': 'Yellow\n .'}, {'role': 'user', 'content': 'Can you explain why bananas turn brown?'}, {'role': 'assistant', 'content': 'Bananas turn brown due to a process called enzymatic browning, which occurs when enzymes in the fruit react with oxygen from the air. This causes the fruit to darken in color and eventually become soft and mushy.'}, {'role': 'user', 'content': 'How can we prevent bananas from turning brown too quickly?'}, {'role': 'assistant', 'content': 'One way to prevent bananas from turning brown too quickly is to store them in a cool, dry place away from direct sunlight. You can also wrap the stem of the banana bunch in plastic wrap, as this will slow down the ripening process.'}, {'role': 'user', 'content': 'Are there any health benefits associated with eating bananas?'}, {'role': 'assistant', 'content': 'Yes, bananas are considered a healthy food choice as they are a good source of potassium, fiber, vitamin C, and vitamin B6. They may also have other health benefits such as p

Number of fcall matches: 1 out of 1 (100.00%)
Number of fcall args matches: 1 out of 1 (100.00%)
Generic LLM accuracy : 57.20%
Average latency per request: 842.02 ms


,Prompt,Actual,Expected,Match
0,"[{'role': 'system', 'content': 'You are a helpful assistant with access to the following functions. Use them if required -\n{\n ""name"": ""calculate_age"",\n ""description"": ""Calculate the age based on the birth date"",\n ""parameters"": {\n ""type"": ""object"",\n ""properties"": {\n ""birth_date"": {\n ""type"": ""string"",\n ""description"": ""The birth date in format YYYY-MM-DD""\n }\n },\n ""required"": [\n ""birth_date""\n ]\n }\n}'}, {'role': 'user', 'content': 'Hi, I was born on 1990-05-15. Can you calculate how old I am?'}, {'role': 'assistant', 'content': {'tool_uses': [{'recipient_name': 'functions.calculate_age', 'parameters': {'birth_date': '1990-05-15'}}]}}, {'role': 'tool', 'content': '{""age"": 31}'}, {'role': 'assistant', 'content': 'Based on the birth date you provided, you are 31 years old.'}, {'role': 'user', 'content': ""That's correct! Thank you.""}]","You're welcome! If you have any other questions, feel free to ask.","You're welcome! If you have any other questions, feel free to ask.",Yes
1,"[{'role': 'system', 'content': 'You are a helpful assistant with access to the following functions. Use them if required -\n{\n ""name"": ""find_similar_movies"",\n ""description"": ""Find similar movies based on a given movie"",\n ""parameters"": {\n ""type"": ""object"",\n ""properties"": {\n ""movie"": {\n ""type"": ""string"",\n ""description"": ""The name of the movie""\n }\n },\n ""required"": [\n ""movie""\n ]\n }\n}'}, {'role': 'user', 'content': 'Hey, I just watched Inception and loved it. Can you suggest similar movies?'}]","{'tool_uses': [{'recipient_name': 'functions.find_similar_movies', 'parameters': {'movie': 'Inception'}}]}","{'tool_uses': [{'recipient_name': 'functions.find_similar_movies', 'parameters': {'movie': 'Inception'}}]}",Yes
2,"[{'role': 'system', 'content': 'You are a helpful assistant with access to the following functions. Use them if required -\n{\n ""name"": ""find_similar_movies"",\n ""description"": ""Find similar movies based on a given movie"",\n ""parameters"": {\n ""type"": ""object"",\n ""properties"": {\n ""movie"": {\n ""type"": ""string"",\n ""description"": ""The name of the movie""\n }\n },\n ""required"": [\n ""movie""\n ]\n }\n}'}, {'role': 'user', 'content': 'Hey, I just watched Inception and loved it. Can you suggest similar movies?'}, {'role': 'assistant', 'content': {'tool_uses': [{'recipient_name': 'functions.find_similar_movies', 'parameters': {'movie': 'Inception'}}]}}, {'role': 'tool', 'content': '{""similar_movies"": [""Interstellar"", ""The Prestige"", ""Memento"", ""Shutter Island"", ""The Matrix""]}'}]","Sure, based on your liking for Inception, you might also enjoy watching these movies: ""Interstellar"", ""The Prestige"", ""Memento"", ""Shutter Island"", and ""The Matrix"".","I found some similar movies to Inception. You might enjoy Interstellar, The Prestige, Memento, Shutter Island, and The Matrix.",No
3,"[{'role': 'system', 'content': 'You are a helpful assistant with access to the following functions. Use them if required -\n{\n ""name"": ""find_similar_movies"",\n ""description"": ""Find similar movies based on a given movie"",\n ""parameters"": {\n ""type"": ""object"",\n ""properties"": {\n ""movie"": {\n ""type"": ""string"",\n ""description"": ""The name of the movie""\n }\n },\n ""required"": [\n ""movie""\n ]\n }\n}'}, {'role': 'user', 'content': 'Hey, I just watched Inception and loved it. Can you suggest similar movies?'}, {'role': 'assistant', 'content': {'tool_uses': [{'recipient_name': 'functions.find_similar_movies', 'parameters': {'movie': 'Inception'}}]}}, {'role': 'tool', 'content': '{""similar_movies"": [""Interstellar"", ""The Prestige"", ""Memento"", ""Shutter Island"", ""The Matrix""]}'}, {'role': 'assistant', 'content': 'Sure, based on your liking for Inception, you might also enjoy watching these movies: ""Interstellar"", ""The Prestige"", ""Memento"", ""Shutter Island"", and ""The Matrix"".'}, {

...

Number of fcall matches: 2 out of 2 (100.00%)
Number of fcall args matches: 2 out of 2 (100.00%)
Generic LLM accuracy : 69.64%
Average latency per request: 1160.15 ms


,Prompt,Actual,Expected,Match
0,"[{'role': 'system', 'content': 'You are a helpful assistant with access to the following functions. Use them if required -\n{\n ""name"": ""get_stock_price"",\n ""description"": ""Get the current stock price"",\n ""parameters"": {\n ""type"": ""object"",\n ""properties"": {\n ""symbol"": {\n ""type"": ""string"",\n ""description"": ""The stock symbol e.g. AAPL""\n }\n },\n ""required"": [\n ""symbol""\n ]\n }\n}\n\n{\n ""name"": ""generate_random_number"",\n ""description"": ""Generate a random number within a specified range"",\n ""parameters"": {\n ""type"": ""object"",\n ""properties"": {\n ""min"": {\n ""type"": ""integer"",\n ""description"": ""The minimum value of the range""\n },\n ""max"": {\n ""type"": ""integer"",\n ""description"": ""The maximum value of the range""\n }\n },\n ""required"": [\n ""min"",\n ""max""\n ]\n }\n}'}, {'role': 'user', 'content': 'Hi, can you tell me the current stock price of Apple?'}, {'role': 'assistant', 'content': {'tool_uses': [{'recipient_name': 'functions.get_stock_price', 'parameters': {'symbol': 'AAPL'}}]}}, {'role': 'tool', 'content': '{""stock_price"": 150.75}'}, {'role': 'assistant', 'content': 'The current stock price of Apple (AAPL) is $150.75.'}, {'role': 'user', 'content': ""That's great! Can you also tell me the current stock price of Microsoft?""}]","{'tool_uses': [{'recipient_name': 'functions.get_stock_price', 'parameters': {'symbol': 'MSFT'}}]}","{'tool_uses': [{'recipient_name': 'functions.get_stock_price', 'parameters': {'symbol': 'MSFT'}}]}",Yes
1,"[{'role': 'system', 'content': 'You are a helpful assistant with access to the following functions. Use them if required -\n{\n ""name"": ""get_stock_price"",\n ""description"": ""Get the current stock price"",\n ""parameters"": {\n ""type"": ""object"",\n ""properties"": {\n ""symbol"": {\n ""type"": ""string"",\n ""description"": ""The stock symbol e.g. AAPL""\n }\n },\n ""required"": [\n ""symbol""\n ]\n }\n}\n\n{\n ""name"": ""generate_random_number"",\n ""description"": ""Generate a random number within a specified range"",\n ""parameters"": {\n ""type"": ""object"",\n ""properties"": {\n ""min"": {\n ""type"": ""integer"",\n ""description"": ""The minimum value of the range""\n },\n ""max"": {\n ""type"": ""integer"",\n ""description"": ""The maximum value of the range""\n }\n },\n ""required"": [\n ""min"",\n ""max""\n ]\n }\n}'}, {'role': 'user', 'content': 'Hi, can you tell me the current stock price of Apple?'}, {'role': 'assistant', 'content': {'tool_uses': [{'recipient_name': 'functions.get_stock_price', 'parameters': {'symbol': 'AAPL'}}]}}, {'role': 'tool', 'content': '{""stock_price"": 150.75}'}, {'role': 'assistant', 'content': 'The current stock price of Apple (AAPL) is $150.75.'}, {'role': 'user', 'content': ""That's great! Can you also tell me the current stock price of Microsoft?""}, {'role': 'assistant', 'content': {'tool_uses': [{'recipient_name': 'functions.get_stock_price', 'parameters': {'symbol': 'MSFT'}}]}}, {'role': 'tool', 'content': '{""stock_price"": 210.22}'}]",The current stock price of Microsoft (MSFT) is $210.22.,The current stock price of Microsoft (MSFT) is $210.22.,Yes
2,"[{'role': 'system', 'content': 'You are a helpful assistant with access to the following functions. Use them if required -\n{\n ""name"": ""get_stock_price"",\n ""description"": ""Get the current stock price"",\n ""parameters"": {\n ""type"": ""object"",\n ""properties"": {\n ""symbol"": {\n ""type"": ""string"",\n ""description"": ""The stock symbol e.g. AAPL""\n }\n },\n ""required"": [\n ""symbol""\n ]\n }\n}\n\n{\n ""name"": ""generate_random_number"",\n ""description"": ""Generate a random number within a specified range"",\n ""parameters"": {\n ""type"": ""object"",\n ""properties"": {\n ""min"": {\n ""type"": ""integer"",\n ""description"": ""The minimum value of the range""\n },\n ""max"": {\n ""type"": ""integer"",\n ""description"": ""The maximum value of the range""\n }\n },\

Number of fcall matches: 1 out of 1 (100.00%)
Number of fcall args matches: 1 out of 1 (100.00%)
Generic LLM accuracy : 63.46%
Average latency per request: 6750.45 ms
615


,Prompt,Actual,Expected,Match
0,"[{'role': 'system', 'content': 'You are a helpful assistant with access to the following functions. Use them if required -\n{\n ""name"": ""create_event"",\n ""description"": ""Create a new event in the calendar"",\n ""parameters"": {\n ""type"": ""object"",\n ""properties"": {\n ""title"": {\n ""type"": ""string"",\n ""description"": ""The title of the event""\n },\n ""start_time"": {\n ""type"": ""string"",\n ""description"": ""The start time of the event""\n },\n ""end_time"": {\n ""type"": ""string"",\n ""description"": ""The end time of the event""\n }\n },\n ""required"": [\n ""title"",\n ""start_time"",\n ""end_time""\n ]\n }\n}\n\n{\n ""name"": ""generate_random_number"",\n ""description"": ""Generate a random number within a given range"",\n ""parameters"": {\n ""type"": ""object"",\n ""properties"": {\n ""min"": {\n ""type"": ""integer"",\n ""description"": ""The minimum value of the range""\n },\n ""max"": {\n ""type"": ""integer"",\n ""description"": ""The maximum value of the range""\n }\n },\n ""required"": [\n ""min"",\n ""max""\n ]\n }\n}'}, {'role': 'user', 'content': 'I need to schedule a meeting for tomorrow.'}]","Sure, I can help with that. Could you please provide me with the title, start time, and end time for the meeting?","Sure, I can help with that. Could you please provide me with the title of the meeting, the start time, and the end time?",Yes
1,"[{'role': 'system', 'content': 'You are a helpful assistant with access to the following functions. Use them if required -\n{\n ""name"": ""create_event"",\n ""description"": ""Create a new event in the calendar"",\n ""parameters"": {\n ""type"": ""object"",\n ""properties"": {\n ""title"": {\n ""type"": ""string"",\n ""description"": ""The title of the event""\n },\n ""start_time"": {\n ""type"": ""string"",\n ""description"": ""The start time of the event""\n },\n ""end_time"": {\n ""type"": ""string"",\n ""description"": ""The end time of the event""\n }\n },\n ""required"": [\n ""title"",\n ""start_time"",\n ""end_time""\n ]\n }\n}\n\n{\n ""name"": ""generate_random_number"",\n ""description"": ""Generate a random number within a given range"",\n ""parameters"": {\n ""type"": ""object"",\n ""properties"": {\n ""min"": {\n ""type"": ""integer"",\n ""description"": ""The minimum value of the range""\n },\n ""max"": {\n ""type"": ""integer"",\n ""description"": ""The maximum value of the range""\n }\n },\n ""required"": [\n ""min"",\n ""max""\n ]\n }\n}'}, {'role': 'user', 'content': 'I need to schedule a meeting for tomorrow.'}, {'role': 'assistant', 'content': 'Sure, I can help with that. Could you please provide me with the title, start time, and end time for the meeting?'}, {'role': 'user', 'content': 'The title is ""Project Discussion"", it should start at 10:00 AM and end at 11:00 AM.'}]","{'tool_uses': [{'recipient_name': 'functions.create_event', 'parameters': {'title': 'Project Discussion', 'start_time': '10:00 AM', 'end_time': '11:00 AM'}}]}","{'tool_uses': [{'recipient_name': 'functions.create_event', 'parameters': {'title': 'Project Discussion', 'start_time': '10:00 AM', 'end_time': '11:00 AM'}}]}",Yes
2,"[{'role': 'system', 'content': 'You are a helpful assistant with access to the following functions. Use them if required -\n{\n ""name"": ""create_event"",\n ""description"": ""Create a new event in the calendar"",\n ""parameters"": {\n ""type"": ""object"",\n ""properties"": {\n ""title"": {\n ""type"": ""string"",\n ""description"": ""The title of the event""\n },\n ""start_time"": {\n ""type"": ""string"",\n ""description"": ""The start time of the event""\n },\n ""end_time"": {\n ""type"": ""string"",\n ""description"": ""The end time of the event""\n }\n },\n ""required"": [\n ""title"",\n ""start_time"",\n ""end_time""\n ]\n }\n}\n\n{\n ""name"": ""generate_random_number"",\n ""description"": ""Generate a random number within a given range"",\n ""parameters"": {\n ""type"": ""object"",\n ""properties"": {\

Number of fcall matches: 1 out of 1 (100.00%)
Number of fcall args matches: 1 out of 1 (100.00%)
Generic LLM accuracy : 84.55%
Average latency per request: 797.20 ms


In [44]:
output_df = pd.concat([df.data for df in eval_baseline_df], ignore_index = True)
print(os.getcwd())
output_df.to_csv("./slm_perf_output.csv", index = False)
output_df.to_json("./slm_perf_output.jsonl", orient = "records", lines = True)

/mnt/batch/tasks/shared/LS_root/mounts/clusters/priyakedia2/code/Users/priyakedia/fine-tune-llama-32b-fc


In [46]:
import pandas as pd

df = pd.read_csv("./slm_perf_output.csv")
df["Prompt"][0]

'[{\'role\': \'system\', \'content\': \'You are a helpful assistant with access to the following functions. Use them if required -\\n{\\n    "name": "search_books",\\n    "description": "Search for books based on title or author",\\n    "parameters": {\\n        "type": "object",\\n        "properties": {\\n            "query": {\\n                "type": "string",\\n                "description": "The search query"\\n            },\\n            "author": {\\n                "type": "string",\\n                "description": "The author of the book"\\n            }\\n        }\\n    }\\n}\\n\\n{\\n    "name": "calculate_tip",\\n    "description": "Calculate the tip amount for a bill",\\n    "parameters": {\\n        "type": "object",\\n        "properties": {\\n            "bill_amount": {\\n                "type": "number",\\n                "description": "The total bill amount"\\n            },\\n            "tip_percentage": {\\n                "type": "number",\\n                